In [9]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import yfinance as yf
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Question 1: Use yfinance to Extract Stock Data for Tesla

In [10]:
# creating a Ticker module object
tesla = yf.Ticker('TSLA')
# fetching the stock data
tesla_data = tesla.history(period = "max")
# reseting the index
tesla_data.reset_index(inplace = True)
# displaying the first five rows
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


# Question 2: Use Webscraping to Extract Tesla Revenue Data for Tesla

In [11]:
# fetching the data
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm" 
html_data = requests.get(url).text

# creating a beautifulSoup object
soup = BeautifulSoup(html_data,'html.parser')

# creating an empty dataframe
tesla_revenue = pd.DataFrame(columns = ['Date','Revenue'])

# populating the dataframe
t = soup.find_all('tbody')[1].find_all('tr')
for row in t:
    data = row.find_all('td')
    date = data[0].text
    revenue = data[1].text
    tesla_revenue = tesla_revenue.append({'Date': date, 'Revenue': revenue}, ignore_index = True)
tesla_revenue.tail()

,Date,Revenue
49,2010-06-30,$28
50,2010-03-31,$21
51,2009-12-31,
52,2009-09-30,$46
53,2009-06-30,$27


In [12]:
tesla_revenue['Revenue'].dropna(inplace = True)
tesla_revenue.tail()

,Date,Revenue
49,2010-06-30,$28
50,2010-03-31,$21
51,2009-12-31,
52,2009-09-30,$46
53,2009-06-30,$27


# Question 3: Use yfinance to Extract Stock Data for GameStop

In [13]:
# creating the Ticker module object
game_stop = yf.Ticker("GME")

# stock data
gme_data = game_stop.history(period = "max")

# reseting the index
gme_data.reset_index(inplace = True)

# displaying first five rows
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620129,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712708,1.716074,1.670626,1.683251,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658001,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578048,1.607505,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662209,1.603296,1.662209,6892800,0.0,0.0


# Question 4: Use Webscraping to Extract GME Revenue Data for GameStop

In [14]:
# requesting the data
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"
html_data2 = requests.get(url).text

# creating BeautifulSoup object
soup2 = BeautifulSoup(html_data2, "html.parser")

# creating the dataframe
gme_revenue = pd.DataFrame(columns = ['Date','Revenue'])

# populating the dataframe
g = soup2.find_all('tbody')[1].find_all('tr')
for row in g:
    data = row.find_all('td')
    date = data[0].text
    revenue = data[1].text
    gme_revenue = gme_revenue.append({'Date':date, 'Revenue':revenue}, ignore_index = True)
gme_revenue.tail()

,Date,Revenue
57,2006-01-31,"$1,667"
58,2005-10-31,$534
59,2005-07-31,$416
60,2005-04-30,$475
61,2005-01-31,$709


# Question 5: Plot Tesla Stock Graph

In [15]:
def make_graph(stock_data, revenue_data, stock):
    revenue_data['Revenue'] = revenue_data['Revenue'].replace('', pd.NA)
    # Drop rows with NaN values in the 'Revenue' column
    revenue_data = revenue_data.dropna(subset=['Revenue'])
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021-06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

In [16]:
tesla_revenue['Revenue'] = tesla_revenue['Revenue'].str.replace("$","")
tesla_revenue['Revenue'] = tesla_revenue['Revenue'].str.replace(",","")
gme_revenue['Revenue'] = gme_revenue['Revenue'].str.replace("$","")
gme_revenue['Revenue'] = gme_revenue['Revenue'].str.replace(",","")

In [17]:
make_graph(tesla_data, tesla_revenue, 'Tesla')

In [18]:
make_graph(gme_data, gme_revenue, "GameStop")